In [1]:
import tensorflow as tf

out_dir = "logs/embeddings"
ckpt_path = tf.train.latest_checkpoint(out_dir)
if not ckpt_path:
    raise FileNotFoundError(f"No checkpoint found under {out_dir}. "
                            "Check the folder and filenames (e.g. embedding.ckpt-1.*).")
print("Using checkpoint:", ckpt_path)

vars_in_ckpt = tf.train.list_variables(ckpt_path)
print("Variables in checkpoint:")
for name, shape in vars_in_ckpt:
    print(" ", name, shape)

candidate = None
candidate_shape = None
largest_size = 0
for name, shape in vars_in_ckpt:
    if len(shape) == 2:
        size = shape[0] * shape[1]
        if size > largest_size:
            largest_size = size
            candidate = name
            candidate_shape = shape

if candidate is None:
    raise RuntimeError("No 2-D variable found in checkpoint. Check saved checkpoint contents above.")

print(f"Loading variable '{candidate}' with shape {candidate_shape} from checkpoint...")
embeddings = tf.train.load_variable(ckpt_path, candidate)  # returns numpy array
print("Loaded embeddings shape:", embeddings.shape)

Using checkpoint: logs/embeddings/embedding.ckpt-1
Variables in checkpoint:
  _CHECKPOINTABLE_OBJECT_GRAPH []
  embedding/.ATTRIBUTES/VARIABLE_VALUE [30, 384]
  save_counter/.ATTRIBUTES/VARIABLE_VALUE []
Loading variable 'embedding/.ATTRIBUTES/VARIABLE_VALUE' with shape [30, 384] from checkpoint...
Loaded embeddings shape: (30, 384)


In [3]:
import faiss
import numpy as np
import pandas as pd

# Cargar metadata
metadata = pd.read_csv("logs/embeddings/metadata.tsv", sep="\t")

# Normalizar embeddings
embeddings_norm = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# Crear índice FAISS
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings_norm.astype("float32"))

def top_k_products(query_idx, k=5):
    query_vec = embeddings_norm[query_idx].astype("float32").reshape(1, -1)
    D, I = index.search(query_vec, k=50)  # pedimos más resultados y luego filtramos

    productos = []
    for dist, idx in zip(D[0], I[0]):
        if metadata.iloc[idx]["tipo"] == "producto":
            productos.append((dist, idx))
        if len(productos) >= k:
            break

    return productos

# Ejemplo: recomendación para el cliente 0
cliente_idx = 5
print("Cliente:", metadata.iloc[cliente_idx]["texto_completo"])
print("\nTop-5 productos similares:")
for dist, idx in top_k_products(cliente_idx, k=5):
    row = metadata.iloc[idx]
    print(f" - {row['label']} | score={dist:.4f}")


Cliente: Luis Hernández Lima Perú Libro: El Círculo Mágico Smartphone Nexus 5G Auriculares Inalámbricos X Camiseta Deportiva Ultralight Zapatillas Urbanas Fit Laptop Gamer Pro Laptop Gamer Pro Laptop Gamer Pro Laptop Gamer Pro Laptop Gamer Pro Smartphone Nexus 5G Una novela de fantasía épica que te sumerge en el reino de Eterea. Sigue la historia de Elara, una joven maga que debe dominar un poder ancestral para salvar a su pueblo de un mal inminente. La autora teje una narrativa intrincada, llena de personajes complejos, criaturas mitológicas y giros inesperados. El mundo está construido con una riqueza de detalles que te hará sentir parte de él. Perfecto para los fans de las sagas de fantasía con temas de heroísmo, traición y redención. El Smartphone Nexus 5G redefine la experiencia móvil. Su pantalla OLED de 6.7 pulgadas con una frecuencia de 120Hz proporciona colores vibrantes y negros profundos. La cámara principal de 108MP, junto con un lente ultra gran angular y un teleobjetivo, 

In [4]:
from sentence_transformers import SentenceTransformer

print("SentenceTransformers disponible. Usando modelo multilingüe.")
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
st_model = SentenceTransformer(model_name)

/Users/erichuiza/Documents/pucp/miería web/dev/sesion-de-clase-06/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SentenceTransformers disponible. Usando modelo multilingüe.


In [5]:
def search_query(query_text, k=5, tipo="producto"):
    # generar embedding del query
    q_emb = st_model.encode([query_text], convert_to_numpy=True)
    q_emb = q_emb / np.linalg.norm(q_emb, axis=1, keepdims=True)

    # buscar en FAISS
    D, I = index.search(q_emb.astype("float32"), k=50)

    resultados = []
    for dist, idx in zip(D[0], I[0]):
        if metadata.iloc[idx]["tipo"] == tipo:
            row = metadata.iloc[idx]
            resultados.append((dist, row["label"], row["texto_completo"]))
        if len(resultados) >= k:
            break

    return resultados

In [8]:
# Ejemplo de uso:
query = "calzado deportivo para correr"
resultados = search_query(query, k=5, tipo="producto")

print(f"🔎 Query: {query}\n")
for score, label, texto in resultados:
    print(f"- {label} | score={score:.4f}\n  {texto[:100]}...")

🔎 Query: calzado deportivo para correr

- Zapatillas Urbanas Fit | score=0.3725
  Zapatillas Urbanas Fit...
- Balón de Fútbol Pro | score=0.1926
  Balón de Fútbol Pro...
- Mochila Urbana Tech | score=0.1872
  Mochila Urbana Tech...
- Set de Ollas Premium | score=0.0929
  Set de Ollas Premium...
- Cámara de Acción 4K | score=0.0877
  Cámara de Acción 4K...
